In [4]:
from dotenv import load_dotenv
import os
from groq import Groq
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

api_key=os.getenv(".env")
#os.environ['GROQ_API_KEY']=getpass.getpass(api_key)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path=r"C:\Users\khmam\Desktop\ExamPrepAI\test_textbook\Compilers Principles, Techniques, & Tools 2nd Ed.pdf"
loader=PyPDFLoader(file_path)

docs=loader.load()
print(len(docs))

In [ ]:
embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2',
        model_kwargs={'device': 'gpu'}
    )

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,chunk_overlap=200,add_start_index=True
)
all_splits=text_splitter.split_documents(docs)
len(all_splits)

In [12]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store=InMemoryVectorStore(embeddings)
ids=vector_store.add_documents(documents=all_splits)

In [ ]:
query="Explain in detail lexical analyser."
results=vector_store.similarity_search(query)
results[0].page_content

In [ ]:
client = Groq(api_key=api_key)
completion = client.chat.completions.create(
    messages=[
        {'role':'system','content':'''Answer the questions from the prompt and the context given by the user. If the answer is not
         found, reply "Cannot provide answer", don't give any additional explanation about the question.Give the answer with proper headings,subheadings and bullet points if it is a long answer.
         You have to help the user understand the answer to the question and format it for notes making.Do not give answers from outside provided context'''},
         {'role':'user','content':f"Context:{results[:5]},Question:{query}"}
    ],
    model="llama3-8b-8192",
)
print(completion.choices[0].message.content)